In [3]:
import numpy as np
import math

In [4]:
def pred_bayes_logistic(w_map, S_N, theta):
    mu_a = np.dot(w_map.T, theta)
    var_a = np.dot(np.dot(theta.T, S_N), theta)
    kappa_var = (1 + math.pi*var_a/8)^(-0.5)
    x = kappa_var*mu_a
    return 1/(1 + np.exp(-x))